<a href="https://colab.research.google.com/github/armandoordonez/eda_couse/blob/main/Basic_Pipeline_custom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# based on  https://towardsdatascience.com/pipeline-and-custom-transformer-with-a-hands-on-case-study-in-python-c416731e6158

# Imports

import pandas as pd
import numpy as np

# Visualización
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

# missing value imputation
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# Supress warnings
import warnings
warnings.filterwarnings("ignore")

In [2]:
data=pd.read_csv("https://raw.githubusercontent.com/armandoordonez/eda_couse/main/data/WA_Fn-UseC_-Telco-Customer-Churn.csv")
data.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [3]:
data['MultipleLines'].value_counts()

No                  3390
Yes                 2971
No phone service     682
Name: MultipleLines, dtype: int64

* Yes ~3.5K records
* No ~ 3K records
* No phone service ~0.5K records

In [8]:
#Transformador personalizado que extrae columnas pasadas como argumento a su constructor

class NoPhoneServiceTransformer():

    #Class Constructor

    def __init__( self, feature_names):
        self._feature_names = feature_names

    #Return self nada más que hacer aquí

    def fit( self, X, y = None ):
        return self

    #M étodo que describe lo que necesitamos que haga este transformador

    def transform(self, X, y = None ):

        def map_values(val):
            if val in ["No phone service"]: return 'No'
            else:
                return val
        X_=X.copy()

        X_[self._feature_names] = X_[self._feature_names].apply(map_values)

        return  X_

In [9]:
data1=[(0, "No"),(0, "Yes"), (1, "No"), (1,"No phone service")]

df = pd.DataFrame(data1, columns=["Label", "MultipleLines"])

df.head()

,Label,MultipleLines
0,0,No
1,0,Yes
2,1,No
3,1,No phone service


In [10]:
pipeline=Pipeline(steps=[("CT", NoPhoneServiceTransformer("MultipleLines"))])
result = pipeline.fit_transform(df)
result

,Label,MultipleLines
0,0,No
1,0,Yes
2,1,No
3,1,No


In [11]:
data['MultipleLines'].value_counts()

No                  3390
Yes                 2971
No phone service     682
Name: MultipleLines, dtype: int64

In [15]:
pipeline=Pipeline(steps=[("CT", NoPhoneServiceTransformer("MultipleLines"))])
result = pipeline.fit_transform(data)
result['MultipleLines'].value_counts()


No     4072
Yes    2971
Name: MultipleLines, dtype: int64